# Projekt - Rachunek prawdopodobieństwa i statystyka
## Analiza rekordów Listy Przebojów Trójki z lat 1982-2020
### Autor: Paweł Kruczkiewicz
#### Data analizy: XX.01.2021

W niniejszej pracy pragnę przyjrzeć się jednej z najbardziej znanych i z pewnością najstarszej liście przebojów w Polsce, tj. Liście Przebojów Trójki. Lista tworzona w Programie Trzecim Polskiego Radia, której pomysłodawcą i głównym prowadzącym i twórcą był Marek Niedźwiecki, przez lata kształtowała gusta muzyczne młodych pokoleń polskich słuchaczy.<br>
Niestety w roku 2020, po tzw. "Aferze w radiowej Trójce" (https://pl.wikipedia.org/wiki/Polskie_Radio_Program_III#Sprawa_pojawienia_si%C4%99_piosenki_%E2%80%9ETw%C3%B3j_b%C3%B3l_jest_lepszy_ni%C5%BC_m%C3%B3j%E2%80%9D_na_Li%C5%9Bcie_Przeboj%C3%B3w), której przyczynkiem było jedno (jak się miało okazać - ostatnie) - notowanie nr 1998, Lista Przebojów Radiowej Trójki została "zawieszona".

Jako wyraz uznania wieloletniej pracy redaktorów Trójki jak i przejaw zwykłej ciekawości na to, jak wygladały gusta "Trójkowiczów" przez wszystkie te lata, chciałbym krótko przeanalizować niemal 2 tysiące notowań

### Krótko o danych
Dane zostały pobrane ze strony https://www.lp3.pl/, skąd zostały "zeskrapowane" z użyciem języka python z pomocą modułu BeautifulSoup4. Pełny kod (wraz z plikiem "requirements.txt") dostępny jest pod podanym linkiem na githubie: https://github.com/pkrucz00/trojkaStats w pliku 